In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from data_generation import DenseGenerator, ChessPositionGen
import keras_tuner as kt

import datetime
%load_ext tensorboard

In [2]:
# Setting paramaters on early stopping
earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=5,
                          verbose=1,
                          mode='min',
                          restore_best_weights=True)

log_dir = "logs/fit/tuned" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [3]:
# Memory management, likely not necessary, but used as a safety as per the documentation recommendations on using GPUS

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


1 Physical GPUs, 1 Logical GPUs


In [4]:
train = pd.read_csv('fens/train.csv')
val = pd.read_csv('fens/val.csv')

In [5]:
# train_dense_gen = DenseGenerator(train, batch_size=1024)
# val_dense_gen = DenseGenerator(val, batch_size=1024)

In [6]:
train_tune_gen = ChessPositionGen(train, batch_size=512)
val_tune_gen = ChessPositionGen(val, batch_size=512)

## Multi Layer Perceptron model for comparison with the CNN versions

This model uses basic densely connected layers and uses a slight variation of the previously used data generator with no reshaping of the array.

In [6]:
dense_model = models.Sequential()
dense_model.add(layers.Dense(832, input_shape=(832,), activation='relu'))
dense_model.add(layers.Dense(64, activation='relu'))
dense_model.add(layers.Dense(64, activation='relu'))
dense_model.add(layers.Dense(1, activation='sigmoid'))
dense_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['acc'])
dense_model.summary()

# Fitting the model
dense_history = dense_model.fit(x=train_dense_gen,
                    validation_data=val_dense_gen,
                    # steps_per_epoch=100,
                    epochs=15,
                    callbacks=[earlystop, tensorboard_callback]
                    )

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 832)               693056    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                53312     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 750,593
Trainable params: 750,593
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
1977/1977 [==============================] - 2590s 1s/step - loss: 0.4226 - acc: 0.8320 - val_loss: 0.3911 - val_acc: 0.8487
Epoch 2/15
1977/1977 [==============================] - 2598s 1s/step - loss: 0.3766 - acc: 0.8544 - val_loss: 0

In [9]:
dense_model.save('MLPmodel-Long-PB')

INFO:tensorflow:Assets written to: DenseModel-PB/assets


Initially this model was only trained for 15 epochs, then when it appeared that it might be roughly comparable with the CNN model, it was trained for another 15 (early stopped after 11, 26 epochs in total) for a more direct comparison.

In [10]:
dense_history = dense_model.fit(x=train_dense_gen,
                    validation_data=val_dense_gen,
                    epochs=15,
                    callbacks=[earlystop, tensorboard_callback])

Epoch 1/15
1977/1977 [==============================] - 2595s 1s/step - loss: 0.2872 - acc: 0.8874 - val_loss: 0.3181 - val_acc: 0.8767
Epoch 2/15
1977/1977 [==============================] - 2590s 1s/step - loss: 0.2852 - acc: 0.8881 - val_loss: 0.3158 - val_acc: 0.8776ETA: 2:49 - l - ETA: 22s
Epoch 3/15
1977/1977 [==============================] - 2582s 1s/step - loss: 0.2833 - acc: 0.8888 - val_loss: 0.3150 - val_acc: 0.8778
Epoch 4/15
1977/1977 [==============================] - 2581s 1s/step - loss: 0.2816 - acc: 0.8895 - val_loss: 0.3151 - val_acc: 0.8781
Epoch 5/15
1977/1977 [==============================] - 2569s 1s/step - loss: 0.2800 - acc: 0.8899 - val_loss: 0.3153 - val_acc: 0.8774
Epoch 6/15
1977/1977 [==============================] - 2583s 1s/step - loss: 0.2784 - acc: 0.8904 - val_loss: 0.3144 - val_acc: 0.8785
Epoch 7/15
1977/1977 [==============================] - 2601s 1s/step - loss: 0.2770 - acc: 0.8909 - val_loss: 0.3159 - val_acc: 0.8780
Epoch 8/15
1977/1977 [==

In [12]:
dense_model.save('MLPmodel-Long.h5')

## Autotuning

For further information on the Keras autotuner, consult the documentation.

In [12]:
def model_builder(hp):
    """
    Autotuner modeling function, based off the CNN model from the full-puzzle-model notebook.
    """
    model = models.Sequential()

    # Set Convolutional layer parameters
    hp_filters = hp.Int('filters', min_value=16, max_value=128, step=8)
    hp_ksize = hp.Int('kernel_size', min_value=2, max_value=8, step=2)

    model.add(layers.Conv2D(filters=hp_filters, kernel_size=hp_ksize, padding='same', input_shape=(8,8,13), activation='relu'))
    model.add(layers.MaxPooling2D(2))
    model.add(layers.Conv2D(filters=hp_filters, kernel_size=hp_ksize, padding='same', activation='relu'))
    model.add(layers.Flatten())

    # Add dense tuning parameters
    hp_units = hp.Int('units', min_value=16, max_value=128, step=8)
    model.add(layers.Dense(units=hp_units, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['acc'])

    return model

tuner = kt.Hyperband(model_builder, objective='val_accuracy', max_epochs=10, directory='tuner', project_name='CNN_tuning')


INFO:tensorflow:Reloading Oracle from existing project tuner/CNN_tuning/oracle.json


With the `model_builder` function and tuner created, let's start the search. The steps per epoch have been reduced in order to finish searching in a reasonable amount of time.

In [13]:
tuner.search(x=train_tune_gen, validation_data=val_tune_gen, steps_per_epoch=1000, callbacks=[earlystop])

best_hps=tuner.get_best_hyperparameters()[0]
print(best_hps)


Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
filters           |88                |?                 
kernel_size       |8                 |?                 
pool_size         |16                |?                 
units             |40                |?                 
tuner/epochs      |2                 |?                 
tuner/initial_e...|0                 |?                 
tuner/bracket     |2                 |?                 
tuner/round       |0                 |?                 

Epoch 1/2


InternalError:  Blas xGEMM launch failed : a.shape=[1,2454,1408], b.shape=[1,1408,40], m=2454, n=40, k=1408
	 [[node sequential/dense/MatMul (defined at /home/nathaniel/anaconda3/envs/Better-learn/lib/python3.8/site-packages/keras_tuner/engine/hypermodel.py:137) ]] [Op:__inference_train_function_1338]

Function call stack:
train_function
